## Collaborative Filtering Using k-Nearest Neighbors (kNN)

Collaborative filtering based on k-nearest neighbors (kNN) kNN is a machine learning algorithm thats finds clusters of similar users based on thier book ratings, and making predictions based on the average of the top (kNN) k-nearest neighbors

K-nearest neighbors (KNN) is a simple, supervised machine learning algorithm that can be used for both classification and regression. This method is easy to implement and understand, but it becomes significantly slow as the size of the data in use increases.

k is a positive integer

N (Nearest)

N (Neighbors)

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

## Importing Datasets

In [2]:
vamsidhar_books = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/books_data.csv')
vamsidhar_ratings  = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/books_ratings_data.csv')
vamsidhar_book_tags = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/book_tags_data.csv')
vamsidhar_tags = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/book_tags_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/vamsidharreddy/CMPE-255-Final-Project/data/books_data.csv'

## Max Columns of data

In [217]:
pd.set_option('display.max_columns', None)

## vamsidhar_books Info

In [218]:
vamsidhar_books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9780439023480.000,Suzanne Collins,2008.000,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.340,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9780439554930.000,"J.K. Rowling, Mary GrandPré",1997.000,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.440,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9780316015840.000,Stephenie Meyer,2005.000,Twilight,"Twilight (Twilight, #1)",en-US,3.570,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9780061120080.000,Harper Lee,1960.000,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.250,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9780743273560.000,F. Scott Fitzgerald,1925.000,The Great Gatsby,The Great Gatsby,eng,3.890,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [219]:
vamsidhar_books.describe()

,id,book_id,best_book_id,work_id,books_count,isbn13,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
count,10000.000,10000.000,10000.000,10000.000,10000.000,9415.000,9979.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000
mean,5000.500,5264696.513,5471213.580,8646183.425,75.713,9755044298883.463,1981.988,4.002,54001.235,59687.322,2919.955,1345.041,3110.885,11475.894,19965.697,23789.806
std,2886.896,7575461.864,7827329.891,11751060.824,170.471,442861920665.574,152.577,0.254,157369.956,167803.785,6124.378,6635.626,9717.124,28546.449,51447.358,79768.886
min,1.000,1.000,1.000,87.000,1.000,195170342.000,-1750.000,2.470,2716.000,5510.000,3.000,11.000,30.000,323.000,750.000,754.000
25%,2500.750,46275.750,47911.750,1008841.000,23.000,9780316192995.000,1990.000,3.850,13568.750,15438.750,694.000,196.000,656.000,3112.000,5405.750,5334.000
50%,5000.500,394965.500,425123.500,2719524.500,40.000,9780451528640.000,2004.000,4.020,21155.500,23832.500,1402.000,391.000,1163.000,4894.000,8269.500,8836.000
75%,7500.250,9382225.250,9636112.500,14517748.250,67.000,9780830777175.000,2011.000,4.180,41053.500,45915.000,2744.250,885.000,2353.250,9287.000,16023.500,17304.500
max,10000.000,33288638.000,35534230.000,56399597.000,3455.000,9790007672390.000,2017.000,4.820,4780653.000,4942365.000,155254.000,456191.000,436802.000,793319.000,1481305.000,3011543.000


## vamsidhar_ratings info

In [220]:
vamsidhar_ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [221]:
vamsidhar_ratings.describe()

,book_id,user_id,rating
count,981756.000,981756.000,981756.000
mean,4943.276,25616.760,3.857
std,2873.207,15228.339,0.984
min,1.000,1.000,1.000
25%,2457.000,12372.000,3.000
50%,4921.000,25077.000,4.000
75%,7414.000,38572.000,5.000
max,10000.000,53424.000,5.000


### Clean the data

In [222]:
vamsidhar_books = vamsidhar_books.dropna()

vamsidhar_ratings = vamsidhar_ratings.sort_values("user_id")
vamsidhar_ratings.drop_duplicates(subset=["user_id","book_id"], keep='first', inplace=True) 
vamsidhar_books.drop_duplicates(subset='original_title', keep='first', inplace=True)

###  Using KNN

In [223]:
vamsidhar_merged_df = pd.merge(books, ratings, how='left', left_on=['id'], right_on=['book_id'])
vamsidhar_df = vamsidhar_merged_df[['id','original_title', 'user_id', 'rating']]

vamsidhar_df = vamsidhar_df.rename(columns = {'id':'book_id'})
vamsidhar_df.head(200)

,book_id,original_title,user_id,rating
0,1,The Hunger Games,314,5
1,1,The Hunger Games,439,3
2,1,The Hunger Games,588,5
3,1,The Hunger Games,1169,4
4,1,The Hunger Games,1185,4
...,...,...,...,...
195,2,Harry Potter and the Philosopher's Stone,51166,5
196,2,Harry Potter and the Philosopher's Stone,51460,4
197,2,Harry Potter and the Philosopher's Stone,51838,4
198,2,Harry Potter and the Philosopher's Stone,52036,3


In [224]:
vamsidhar_df.describe()

,book_id,user_id,rating
count,763421.000,763421.000,763421.000
mean,4657.949,25132.162,3.852
std,2875.106,15229.804,0.981
min,1.000,1.000,1.000
25%,2123.000,11843.000,3.000
50%,4509.000,24230.000,4.000
75%,7087.000,38032.000,5.000
max,9999.000,53424.000,5.000


In [225]:
vamsidhar_ratings = df.pivot_table(index='book_id',columns='user_id',values='rating').fillna(0)

pd.set_option('display.max_columns', 100)
vamsidhar_ratings.head()

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,53375,53376,53377,53378,53379,53380,53381,53382,53383,53384,53385,53386,53387,53388,53389,53390,53391,53392,53393,53394,53395,53396,53397,53398,53399,53400,53401,53402,53403,53404,53405,53406,53407,53408,53409,53410,53411,53412,53413,53414,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [226]:
vamsidhar_ratings.shape

(7755, 50681)

In [227]:
vamsidhar_ratings_matrix = csr_matrix(ratings_df.values)

In [228]:
vamsidhar_model_knn = NearestNeighbors(metric='cosine', algorithm = 'brute')
vamsidhar_model_knn.fit(ratings_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

### Some Helper Functions

In [229]:
def get_book_id(book_title):
    vamsi_target_df = df.loc[df['original_title'] == book_title]
    return vamsi_target_df['book_id'].iloc[0]

id_BayouMoon = get_book_id('Bayou Moon')
print(id_BayouMoon)

9996


In [230]:
def get_title(book_id):
    vamsi_target_df = df.loc[df['book_id'] == book_id]
    return vamsi_target_df['original_title'].iloc[0]

print(get_title(1))

The Hunger Games


In [231]:
def get_recomm(book_title, num_neighbors=10, display=False): 
    vamsi_book_ids = []
    
    query_index = get_book_id(book_title) - 1
    
    if num_neighbors > 0:
        distances, indices = model_knn.kneighbors(ratings_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = num_neighbors + 1)
    else:
        distances, indices = model_knn.kneighbors(ratings_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10 + 1)
    
    for i in range(0, len(distances.flatten())):
        if display is True:
            if i == 0:
                print('Recommendations for ', book_title, '\n')
            else:    
                print('{0}\t Book ID: {1}\t  Distance: {2}:\n'.format(i, ratings_df.index[indices.flatten()[i]], distances.flatten()[i]))
        
        vamsi_book_ids.append(ratings_df.index[indices.flatten()[i]])
    
    return vamsi_book_ids

### Test the Results

### Top 15 recommendations for Harry Potter and the Philosopher's Stone

In [232]:
recommendations_for_HarryPotterandthePhilosophersStone = get_recomm("Harry Potter and the Philosopher's Stone", num_neighbors=15, display=True)

Recommendations for  Harry Potter and the Philosopher's Stone 

1	 Book ID: 4	  Distance: 0.3892592020883805:

2	 Book ID: 33	  Distance: 0.40241308954159594:

3	 Book ID: 13	  Distance: 0.4097614772716248:

4	 Book ID: 5	  Distance: 0.41205601974725015:

5	 Book ID: 19	  Distance: 0.4146445358053812:

6	 Book ID: 28	  Distance: 0.41542064436796167:

7	 Book ID: 18	  Distance: 0.4226021355475864:

8	 Book ID: 7	  Distance: 0.42733371842918766:

9	 Book ID: 15	  Distance: 0.4317700868712365:

10	 Book ID: 43	  Distance: 0.4402457989008367:

11	 Book ID: 1	  Distance: 0.4444738573252147:

12	 Book ID: 24	  Distance: 0.44452856661417617:

13	 Book ID: 23	  Distance: 0.4454417431428892:

14	 Book ID: 10	  Distance: 0.4538274129514055:

15	 Book ID: 14	  Distance: 0.4562715283856553:



In [233]:
for b in recommendations_for_HarryPotterandthePhilosophersStone[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

id: 4 		Book:  To Kill a Mockingbird
id: 33 		Book:  Memoirs of a Geisha
id: 13 		Book:  Nineteen Eighty-Four
id: 5 		Book:  The Great Gatsby
id: 19 		Book:   The Fellowship of the Ring
id: 28 		Book:  Lord of the Flies 
id: 18 		Book:  Harry Potter and the Prisoner of Azkaban
id: 7 		Book:  The Hobbit or There and Back Again
id: 15 		Book:  Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
id: 43 		Book:  Jane Eyre
id: 1 		Book:  The Hunger Games
id: 24 		Book:  Harry Potter and the Goblet of Fire
id: 23 		Book:  Harry Potter and the Chamber of Secrets
id: 10 		Book:  Pride and Prejudice
id: 14 		Book:  Animal Farm: A Fairy Story


### Top 15 recommendations for To Kill a Mockingbird

In [234]:
book_ids_for_H = get_recomm('To Kill a Mockingbird', num_neighbors=15)
for b in book_ids_for_H[1:]:
    print(get_title(b))

The Great Gatsby
The Catcher in the Rye
Harry Potter and the Philosopher's Stone
Of Mice and Men 
Pride and Prejudice
Animal Farm: A Fairy Story
Memoirs of a Geisha
Nineteen Eighty-Four
Little Women
Lord of the Flies 
Jane Eyre
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
The Lion, the Witch and the Wardrobe
The Scarlet Letter
The Grapes of Wrath


### Top 20 recommendations for The Great Gatsby

In [235]:
book_ids_for_H = get_recomm('The Great Gatsby', num_neighbors=20)
for b in book_ids_for_H[1:]:
    print(get_title(b))

To Kill a Mockingbird
The Catcher in the Rye
Harry Potter and the Philosopher's Stone
Memoirs of a Geisha
Animal Farm: A Fairy Story
Little Women
Of Mice and Men 
The Scarlet Letter
Jane Eyre
Nineteen Eighty-Four
Pride and Prejudice
Lord of the Flies 
The Hobbit or There and Back Again
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
Män som hatar kvinnor
The Lovely Bones
Brave New World
The Grapes of Wrath
An Excellent conceited Tragedie of Romeo and Juliet
 The Fellowship of the Ring
